# Evaluating multiple feature recipes on the CPAC_N7_09_15_20 dataset

- **Model**: Gradient Boosted Trees (histogram-based)
- **Target(s)**: `TF_Pelvis_Moment_X_BWBH` (`TF_Pelvis_Moment_Y_BWBH`)
- **Features**: various (approx. 63 alternative _recipes_)
- **Results**: $r^2$ scores, feature importances (permutation-based)
- **Evaluation strategy**: cross-validation (leave one subject out)

## Libraries

In [1]:
# Standard library
import warnings
import os


# Third party
import numpy as np
import pandas as pd
import sklearn
assert sklearn.__version__ >= "0.21", "Use the conda_python3_latest kernel!"
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn import (ensemble, metrics, preprocessing, 
                     pipeline, inspection, model_selection)

from IPython.display import display, Markdown


# Local
import utils

## Load Dataset

In [2]:
df_orig = utils.load_dataset("s3://cpac/ORIG/CPAC_N7_09_15_20/CPAC10S_N7_09_15_20.csv")
df_orig.describe()

,M_Trial_Num,M_Mass,M_Mass_to_L5S1,M_sub_task_indices,M_sub_task_num,M_include_overall,M_Index,M_Sub,M_sub_task_num_overall,M_Index_overall,...,RWEO_03_04_00_00_INSOLE_LY_AP_threshF50_mm,RWEO_01_00_00_00_INSOLE_RFORCE_threshF50_N,RWEO_01_02_00_00_INSOLE_RX_ML_threshF50_mm,RWEO_01_02_00_00_INSOLE_RY_AP_threshF50_mm,RWEF_03_00_00_00_INSOLE_LFORCE_threshF50_BW,RWEF_03_04_00_00_INSOLE_LX_ML_threshF50_BH,RWEF_03_04_00_00_INSOLE_LY_AP_threshF50_BH,RWEF_01_00_00_00_INSOLE_RFORCE_threshF50_BW,RWEF_01_02_00_00_INSOLE_RX_ML_threshF50_BH,RWEF_01_02_00_00_INSOLE_RY_AP_threshF50_BH
count,1.366326e+06,1.366326e+06,1.040590e+06,1.366326e+06,1.366326e+06,1.366326e+06,1.366326e+06,1.366326e+06,1.366326e+06,1.366326e+06,...,1.178199e+06,1.363161e+06,1.223529e+06,1.223529e+06,1.363161e+06,1.178199e+06,1.178199e+06,1.363161e+06,1.223529e+06,1.223529e+06
mean,6.907848e+01,1.059573e+01,3.126288e-01,2.912306e+02,5.583528e+00,8.073256e-01,2.716548e+03,3.855173e+00,2.384674e+02,8.848465e+04,...,1.216852e+02,3.945501e+02,4.903974e+01,1.334307e+02,6.155901e-01,2.797499e-02,6.701906e-02,6.472693e-01,2.697077e-02,7.335476e-02
std,2.360488e+01,5.886671e+00,1.644402e-01,3.589143e+02,4.895601e+00,3.943996e-01,2.598654e+03,1.998726e+00,1.246093e+02,6.576090e+04,...,5.277402e+01,2.768346e+02,8.572305e+00,4.881799e+01,5.533887e-01,5.171558e-03,2.932588e-02,5.626440e-01,4.770854e-03,2.717074e-02
min,1.000000e+00,0.000000e+00,5.026420e-02,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.950165e-01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.900000e+01,5.000000e+00,1.602113e-01,1.160000e+02,1.000000e+00,1.000000e+00,5.950000e+02,2.000000e+00,1.280000e+02,2.824900e+04,...,7.683000e+01,1.603150e+02,4.502000e+01,9.433000e+01,2.381477e-01,2.584712e-02,4.235504e-02,2.565648e-01,2.446361e-02,5.173437e-02
50%,7.900000e+01,1.000000e+01,2.776286e-01,2.350000e+02,4.000000e+00,1.000000e+00,1.853500e+03,4.000000e+00,2.500000e+02,8.517900e+04,...,1.140600e+02,3.650450e+02,5.061000e+01,1.317400e+02,5.112327e-01,2.904687e-02,6.220556e-02,5.398100e-01,2.793333e-02,7.169364e-02
75%,8.700000e+01,1.500000e+01,4.583651e-01,3.790000e+02,8.000000e+00,1.000000e+00,4.185000e+03,6.000000e+00,3.430000e+02,1.421090e+05,...,1.667400e+02,5.945630e+02,5.464000e+01,1.751000e+02,8.329281e-01,3.126012e-02,9.096354e-02,8.659949e-01,2.999444e-02,9.523958e-02
max,9.600000e+01,2.300000e+01,7.376715e-01,7.238000e+03,2.200000e+01,1.000000e+00,1.411900e+04,7.000000e+00,4.570000e+02,2.370080e+05,...,2.477000e+02,1.558065e+03,7.442000e+01,2.508700e+02,5.463242e+00,4.218023e-02,1.440116e-01,5.719531e+00,4.326744e-02,1.458547e-01


## Associate column names

In [3]:
def _get_columns_with_prefix(df, prefix):
    columns = []
    for column in df.columns:
        if column.startswith(prefix):
            columns.append(column)
    return columns
    
def get_target_names(df):
    return _get_columns_with_prefix(df, "T_")

def get_meta_names(df):
    return _get_columns_with_prefix(df, "M_")    

## Clean-up dataset

- Remove samples based on `M_include_overall`

In [4]:
df = df_orig[df_orig["M_include_overall"] > 0]
print(f"Number of samples: {df.shape[0]:,d} (before clean-up: {df_orig.shape[0]:,d})")
print(f"Number of trials: {len(df['M_Trial_Name'].unique())} (before clean-up: {len(df_orig['M_Trial_Name'].unique())})")
print(f"Number of subjects: {len(df['M_Sub'].unique())}")

Number of samples: 1,103,070 (before clean-up: 1,366,326)
Number of trials: 162 (before clean-up: 162)
Number of subjects: 7


## Predictor configurations (recipes)

In [5]:
def predictor_short_name(predictor):
    return predictor[17:]

def predictor_sensor_number(predictor):
    return int(predictor[5:7])

def filter_predictors(all_predictors, patterns):
    if isinstance(patterns, str):
        patterns = (patterns,)
        
    predictors = []
    for predictor in all_predictors:
        for pattern in patterns:
            if pattern in predictor:
                predictors.append(predictor)
                break
    return predictors


feature_sets = {

    "Recipe 1: insole,foot IMUs,shank IMUs,thigh IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '07_08_00_00', '10_11_00_00', '06_07_00_00', '09_10_00_00', '05_06_00_00', '05_09_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_08_00_00', '05_11_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', '01_03_00_00', '05_12_01_03', '05_06_01_03', '05_09_01_03', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 2: insole,foot IMUs,shank IMUs,thigh IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '06_07_00_00', '09_10_00_00', '05_06_00_00', '05_09_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_08_00_00', '05_11_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', '01_03_00_00', '05_06_01_03', '05_09_01_03', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 3: insole,foot IMUs,shank IMUs,thigh IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '06_07_00_00', '09_10_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 4: insole,foot IMUs,shank IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '07_08_00_00', '10_11_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_08_00_00', '05_11_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', '01_03_00_00', '05_12_01_03', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 5: insole,foot IMUs,thigh IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '05_06_00_00', '05_09_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_08_00_00', '05_11_00_00', '05_00_00_00', '01_03_00_00', '05_12_01_03', '05_06_01_03', '05_09_01_03', 'FOOT_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 6: insole,shank IMUs,thigh IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '06_07_00_00', '09_10_00_00', '05_06_00_00', '05_09_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', '01_03_00_00', '05_12_01_03', '05_06_01_03', '05_09_01_03', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 7: foot IMUs,shank IMUs,thigh IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '07_08_00_00', '10_11_00_00', '06_07_00_00', '09_10_00_00', '05_06_00_00', '05_09_00_00', '05_08_00_00', '05_11_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 8: insole,foot IMUs,shank IMUs,thigh IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '06_07_00_00', '09_10_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 9: insole,foot IMUs,shank IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_08_00_00', '05_11_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', '01_03_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL')
        ),

    "Recipe 10: insole,foot IMUs,shank IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 11: insole,foot IMUs,thigh IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_06_00_00', '05_09_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_08_00_00', '05_11_00_00', '05_00_00_00', '01_03_00_00', '05_06_01_03', '05_09_01_03', 'FOOT_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 12: insole,foot IMUs,thigh IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'FOOT_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 13: insole,foot IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_08_00_00', '05_11_00_00', '05_00_00_00', '01_03_00_00', '05_12_01_03', 'FOOT_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 14: insole,shank IMUs,thigh IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('06_07_00_00', '09_10_00_00', '05_06_00_00', '05_09_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', '01_03_00_00', '05_06_01_03', '05_09_01_03', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 15: insole,shank IMUs,thigh IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('06_07_00_00', '09_10_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 16: insole,shank IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', '01_03_00_00', '05_12_01_03', 'SHANK_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 17: insole,thigh IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '05_06_00_00', '05_09_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_00_00_00', '01_03_00_00', '05_12_01_03', '05_06_01_03', '05_09_01_03', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 18: foot IMUs,shank IMUs,thigh IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '06_07_00_00', '09_10_00_00', '05_06_00_00', '05_09_00_00', '05_08_00_00', '05_11_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 19: foot IMUs,shank IMUs,thigh IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '06_07_00_00', '09_10_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 20: foot IMUs,shank IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '07_08_00_00', '10_11_00_00', '05_08_00_00', '05_11_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 21: foot IMUs,thigh IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '05_06_00_00', '05_09_00_00', '05_08_00_00', '05_11_00_00', '05_00_00_00', 'FOOT_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 22: shank IMUs,thigh IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '06_07_00_00', '09_10_00_00', '05_06_00_00', '05_09_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 23: insole,foot IMUs,shank IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL')
        ),

    "Recipe 24: insole,foot IMUs,thigh IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'FOOT_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 25: insole,foot IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_08_00_00', '05_11_00_00', '05_00_00_00', '01_03_00_00', 'FOOT_ANGLE_VL')
        ),

    "Recipe 26: insole,foot IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'FOOT_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 27: insole,shank IMUs,thigh IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('06_07_00_00', '09_10_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 28: insole,shank IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', '01_03_00_00', 'SHANK_ANGLE_VL')
        ),

    "Recipe 29: insole,shank IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'SHANK_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 30: insole,thigh IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_06_00_00', '05_09_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_00_00_00', '01_03_00_00', '05_06_01_03', '05_09_01_03', 'THIGH_ANGLE_VL')
        ),

    "Recipe 31: insole,thigh IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 32: insole,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_00_00_00', '01_03_00_00', '05_12_01_03', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 33: foot IMUs,shank IMUs,thigh IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '06_07_00_00', '09_10_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 34: foot IMUs,shank IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', '05_08_00_00', '05_11_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL')
        ),

    "Recipe 35: foot IMUs,shank IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 36: foot IMUs,thigh IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_06_00_00', '05_09_00_00', '05_08_00_00', '05_11_00_00', '05_00_00_00', 'FOOT_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 37: foot IMUs,thigh IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('FOOT_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 38: foot IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '05_08_00_00', '05_11_00_00', '05_00_00_00', 'FOOT_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 39: shank IMUs,thigh IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('06_07_00_00', '09_10_00_00', '05_06_00_00', '05_09_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 40: shank IMUs,thigh IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('06_07_00_00', '09_10_00_00', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 41: shank IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '05_07_00_00', '05_10_00_00', '05_00_00_00', 'SHANK_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 42: thigh IMUs,pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '05_06_00_00', '05_09_00_00', '05_00_00_00', 'THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 43: insole,foot IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'FOOT_ANGLE_VL')
        ),

    "Recipe 44: insole,shank IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'SHANK_ANGLE_VL')
        ),

    "Recipe 45: insole,thigh IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'THIGH_ANGLE_VL')
        ),

    "Recipe 46: insole,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '05_00_00_00', '01_03_00_00')
        ),

    "Recipe 47: insole,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 48: foot IMUs,shank IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('07_08_00_00', '10_11_00_00', 'FOOT_ANGLE_VL', 'SHANK_ANGLE_VL')
        ),

    "Recipe 49: foot IMUs,thigh IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('FOOT_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 50: foot IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_08_00_00', '05_11_00_00', '05_00_00_00', 'FOOT_ANGLE_VL')
        ),

    "Recipe 51: foot IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('FOOT_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 52: shank IMUs,thigh IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('06_07_00_00', '09_10_00_00', 'SHANK_ANGLE_VL', 'THIGH_ANGLE_VL')
        ),

    "Recipe 53: shank IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_07_00_00', '05_10_00_00', '05_00_00_00', 'SHANK_ANGLE_VL')
        ),

    "Recipe 54: shank IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('SHANK_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 55: thigh IMUs,pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_06_00_00', '05_09_00_00', '05_00_00_00', 'THIGH_ANGLE_VL')
        ),

    "Recipe 56: thigh IMUs,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('THIGH_ANGLE_VL', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 57: pelvis IMU,trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_12_00_00', '05_00_00_00', 'TRUNK_ANGLE_VL')
        ),

    "Recipe 58: insole":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('01_00_00_00', '03_00_00_00', '01_02_00_00', '03_04_00_00', '01_03_00_00')
        ),

    "Recipe 59: foot IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('FOOT_ANGLE_VL',)
        ),

    "Recipe 60: shank IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('SHANK_ANGLE_VL',)
        ),

    "Recipe 61: thigh IMUs":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('THIGH_ANGLE_VL',)
        ),

    "Recipe 62: pelvis IMU":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('05_00_00_00',)
        ),

    "Recipe 63: trunk":
        filter_predictors(
            filter_predictors(df.columns, ("SWRF", "SWEF")),
            ('TRUNK_ANGLE_VL',)
        )    
}

for feature_set_name, predictors in feature_sets.items():
    sensors = set(map(predictor_sensor_number, predictors))
    print(f"{feature_set_name}\n\tPredictors: {len(predictors)}, Sensors: {len(sensors)}\n")

Recipe 1: insole,foot IMUs,shank IMUs,thigh IMUs,pelvis IMU,trunk
	Predictors: 80, Sensors: 7

Recipe 2: insole,foot IMUs,shank IMUs,thigh IMUs,pelvis IMU
	Predictors: 58, Sensors: 7

Recipe 3: insole,foot IMUs,shank IMUs,thigh IMUs,trunk
	Predictors: 48, Sensors: 7

Recipe 4: insole,foot IMUs,shank IMUs,pelvis IMU,trunk
	Predictors: 54, Sensors: 5

Recipe 5: insole,foot IMUs,thigh IMUs,pelvis IMU,trunk
	Predictors: 62, Sensors: 3

Recipe 6: insole,shank IMUs,thigh IMUs,pelvis IMU,trunk
	Predictors: 68, Sensors: 5

Recipe 7: foot IMUs,shank IMUs,thigh IMUs,pelvis IMU,trunk
	Predictors: 57, Sensors: 5

Recipe 8: insole,foot IMUs,shank IMUs,thigh IMUs
	Predictors: 41, Sensors: 7

Recipe 9: insole,foot IMUs,shank IMUs,pelvis IMU
	Predictors: 32, Sensors: 5

Recipe 10: insole,foot IMUs,shank IMUs,trunk
	Predictors: 36, Sensors: 5

Recipe 11: insole,foot IMUs,thigh IMUs,pelvis IMU
	Predictors: 40, Sensors: 3

Recipe 12: insole,foot IMUs,thigh IMUs,trunk
	Predictors: 30, Sensors: 3

Recipe 1

## Train and evaluate boosted tree models

In [8]:
def evaluate(target_name, feature_names):
    X, y, groups = df[feature_names], df[target_name], df["M_Sub"]
    
    model = pipeline.Pipeline([
        ('scaler', preprocessing.StandardScaler()),
        ('gboost', ensemble.HistGradientBoostingRegressor())
    ])
    
    logo = model_selection.LeaveOneGroupOut()
    r2_score = model_selection.cross_val_score(
        model, X, y, cv=logo, groups=groups, n_jobs=-1).mean()
    
    # Feature importances on the full training set
    model.fit(X, y)
    perm_imp = inspection.permutation_importance(model, X, y, n_repeats=5, n_jobs=-1)
    importance = pd.Series(perm_imp.importances_mean, index=X.columns)
    importance.sort_values(ascending=False, inplace=True)

    return r2_score, importance

def run_experiments(target_name, feature_sets):
    warnings.filterwarnings('ignore')
    r2_scores = {}
    importances = {}
    
    for feature_set_name, feature_names in feature_sets.items():
        r2_score, importance = evaluate(target_name, feature_names)
        r2_scores[feature_set_name] = r2_score
        importances[feature_set_name] = importance
        display(
            Markdown(
                "---\n"
                f"**Target**: {target_name}  \n"
                f"**Features**: {feature_set_name}  \n"
                f"**$R^2$ = {r2_score:.3f}**"
            )
        )
                                        
    warnings.filterwarnings('default')
    return r2_scores, importances


## Run experiments, save data

In [11]:
target_name = "TF_Pelvis_Moment_Y_BWBH"
r2_scores, importances = run_experiments(target_name, feature_sets)

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 1: insole,foot IMUs,shank IMUs,thigh IMUs,pelvis IMU,trunk  
**$R^2$ = 0.872**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 2: insole,foot IMUs,shank IMUs,thigh IMUs,pelvis IMU  
**$R^2$ = 0.850**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 3: insole,foot IMUs,shank IMUs,thigh IMUs,trunk  
**$R^2$ = 0.851**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 4: insole,foot IMUs,shank IMUs,pelvis IMU,trunk  
**$R^2$ = 0.859**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 5: insole,foot IMUs,thigh IMUs,pelvis IMU,trunk  
**$R^2$ = 0.870**

...

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 59: foot IMUs  
**$R^2$ = 0.450**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 60: shank IMUs  
**$R^2$ = 0.424**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 61: thigh IMUs  
**$R^2$ = 0.477**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 62: pelvis IMU  
**$R^2$ = 0.519**

---
**Target**: TF_Pelvis_Moment_Y_BWBH  
**Features**: Recipe 63: trunk  
**$R^2$ = 0.638**

In [12]:
os.makedirs("results", exist_ok=True)
with pd.ExcelWriter(f"results/CPAC10S_N7_09_15_20 - {target_name}.xlsx") as writer:
    df_results = pd.DataFrame({"R2 Score": r2_scores})
    df_results.to_excel(writer, sheet_name="R2 Score")
    
    for feature_set_name, importance in importances.items():
        df_results = pd.DataFrame(
            {
                "Short name": map(predictor_short_name, importance.index),
                "Importance": importance,
            }
        )
        df_results.to_excel(writer, sheet_name=feature_set_name.split(":")[0])